In [2]:
import sys
import os

# 현재 작업 디렉토리 확인
current_dir = os.getcwd()
print("Current directory:", current_dir)

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.append(project_root)

Current directory: c:\Users\qtly_u\Desktop\image_processing\image_processing


In [16]:
import os
import cv2
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
from func.func import *
import json
import ast
from skimage import feature

In [4]:
with open('increased_id.json', 'r') as file:
    increased_id = json.load(file)
with open('decreased_id.json', 'r') as file:
    decreased_id = json.load(file)

In [5]:
# JSON 파일 경로
json_file_path = 'result_after_processing.json'

# JSON 파일 읽기
with open(json_file_path, 'r') as f:
    result = json.load(f)

In [6]:
len(result)

11737

In [7]:
center = []
image_id = []
score = []

for item in result:
    center.append(item['center'])
    image_id.append(item['image_id'])
    score.append(item['score'])

result_df = pd.DataFrame({'image_id':image_id,'center':center, 'score':score})
result_df

,image_id,center,score
0,136,"[33.959999084472656, 217.1750030517578]",0.615619
1,136,"[27.110000610351562, 243.05999755859375]",0.548477
2,397,"[116.06999969482422, 102.16500091552734]",0.422394
3,443,"[377.0249938964844, 236.80499267578125]",0.445117
4,443,"[73.57499694824219, 308.0450134277344]",0.430084
...,...,...,...
11732,581674,"[159.0500030517578, 222.47999572753906]",0.432098
11733,581683,"[403.7650146484375, 490.3500061035156]",0.601338
11734,581683,"[381.5, 430.114990234375]",0.467200
11735,581880,"[111.37000274658203, 122.22000122070312]",0.273316


result_df에 bbox area 붙이려고 함

In [8]:
# JSON 파일 읽기
with open('..\merged_keypoints.json', 'r') as val_file:
    annotation = json.load(val_file)

In [9]:
annotation['annotations'][0].keys()

dict_keys(['segmentation', 'num_keypoints', 'area', 'iscrowd', 'keypoints', 'image_id', 'bbox', 'category_id', 'id'])

In [14]:
image_id = []
bbox = []
id = []

for item in annotation['annotations']:
    image_id.append(item['image_id'])
    bbox.append(item['bbox'])
    id.append(item['id'])

In [15]:
annotation_df = pd.DataFrame({'image_id':image_id, 'id': id, 'bbox':bbox})

annotation_df["width"] = annotation_df["bbox"].apply(lambda bbox: bbox[2])
annotation_df["height"] = annotation_df["bbox"].apply(lambda bbox: bbox[3])
annotation_df["bbox_size"] = annotation_df["width"]*annotation_df["height"]

In [19]:
# bbox 열을 기반으로 x, y, width, height 열을 생성
annotation_df[['x', 'y', 'width', 'height']] = pd.DataFrame(annotation_df['bbox'].tolist(), index=annotation_df.index)

# _xywh2cs 함수를 이용하여 center, scale 계산
annotation_df[['center', 'scale']] = annotation_df.apply(lambda row: pd.Series(_xywh2cs(row['x'], row['y'], row['width'], row['height'])), axis=1)


print(annotation_df)

       image_id       id                             bbox   width  height  \
0           136   500146     [0.84, 114.71, 53.54, 257.7]   53.54  257.70   
1           136   502933        [0, 61.35, 68.92, 312.65]   68.92  312.65   
2           397  2151765        [3.74, 0, 225.66, 205.33]  225.66  205.33   
3           443   184971   [117.57, 1.08, 519.91, 472.45]  519.91  472.45   
4           443   224310    [488.63, 2.16, 151.37, 303.1]  151.37  303.10   
...         ...      ...                              ...     ...     ...   
15514    581674  1746490  [133.43, 164.25, 52.24, 117.46]   52.24  117.46   
15515    581683   445516  [380.57, 428.83, 47.39, 124.04]   47.39  124.04   
15516    581683  1222053       [359.5, 384.93, 45, 91.37]   45.00   91.37   
15517    581880   456253    [3.74, 20.95, 216.26, 203.54]  216.26  203.54   
15518    581887  1709172   [533.14, 351.47, 51.64, 78.88]   51.64   78.88   

         bbox_size       x       y                  center  \
0       13797

In [26]:
annotation_df = annotation_df.sort_values(by='image_id', ascending=True).reset_index(drop=True)

# 두 개의 열을 기준으로 정렬 (예: Column1을 오름차순으로 정렬하고, 그 후에 Column2를 오름차순으로 정렬)
result_df = result_df.sort_values(by=['image_id', 'score'], ascending=[True, True]).reset_index(drop=True)

In [28]:
from scipy.spatial.distance import cdist

def find_closest_gt(row, gt_df):
    if not pd.isna(row['center']).any():
        center = row['center']
        gt_for_image = gt_df[gt_df['image_id'] == row['image_id']]
        distances = cdist([center], gt_for_image['center'].tolist())
        closest_index = distances.argmin()
        closest_gt_row = gt_for_image.iloc[closest_index]
        return closest_gt_row
    else:
        return pd.Series([pd.NA] * len(gt_df.columns))

#result_df의 각 행에 대해 가장 가까운 gt_df의 값을 찾아서 병합
df = pd.concat([result_df, result_df.apply(find_closest_gt, gt_df=annotation_df, axis=1)], axis=1)

In [31]:
df.columns

Index(['image_id', 'center', 'score', 'image_id', 'id', 'bbox', 'width',
       'height', 'bbox_size', 'x', 'y', 'center', 'scale'],
      dtype='object')

In [35]:
# 중복된 열 이름 확인
duplicated_columns = df.columns[df.columns.duplicated()].tolist()

# 이름이 같은 열 중 첫 번째 열만 남기고 나머지는 제거
df = df.loc[:, ~df.columns.duplicated()]

## metric 계산 적용

In [36]:
entropy = []
laplacian = []
brightness = []
B_sd = []
c_moment = []
contra = []
img_size = []
edge_intensity = []
edge_num = []
sift = []
psnr = []
centroid_l = []

for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    height, width= image.shape
    img_size_value = width * height

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    entropy_value = calculate_image_entropy_(cropped_image)
    laplacian_value = calculate_laplacian_variance_(cropped_image)
    brightness_value = np.mean(cropped_image)
    sd_value = np.std(cropped_image)
    c_moment_value = moment(cropped_image)
    contrast_value = calculate_contrast(cropped_image)
    edge_num_value, edge_value = edge(cropped_image)
    SIFT_count_value = SIFT_count(cropped_image)
    psnr_value = calculate_psnr(cropped_image)
    cx, cy = c_moment_value
    centroid_l_value = centroid_degree(w, h, cx, cy)

    entropy.append(entropy_value)
    laplacian.append(laplacian_value)
    brightness.append(brightness_value)
    B_sd.append(sd_value)
    c_moment.append(c_moment_value)
    contra.append(contrast_value)
    img_size.append(img_size_value)
    edge_intensity.append(edge_value)
    edge_num.append(edge_num_value)
    sift.append(SIFT_count_value)
    psnr.append(psnr_value)
    centroid_l.append(centroid_l_value)

KeyError: 'image_file'

In [ ]:
# RGB
red = []
green = []
blue = []
color_var = []

# Not Grayscale
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    
    # 이미지 배열에서 각 채널의 값 가져오기
    red_channel = cropped_image[:, :, 0]
    green_channel = cropped_image[:, :, 1]
    blue_channel = cropped_image[:, :, 2]

    # 각 채널별 평균과 분산 계산
    red_mean, red_variance = np.mean(red_channel), np.var(red_channel)
    green_mean, green_variance = np.mean(green_channel), np.var(green_channel)
    blue_mean, blue_variance = np.mean(blue_channel), np.var(blue_channel)
    variance = np.mean([red_variance,green_variance,blue_variance])

    red.append(red_mean)
    green.append(green_mean)
    blue.append(blue_mean)
    color_var.append(variance)

In [ ]:
# HSV
saturation = []

# Not Grayscale
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]

    saturation_value = np.mean(cropped_image[:, :, 1])

    saturation.append(saturation_value)

In [ ]:
df['entropy'] = entropy
df['laplacian'] = laplacian
df['brightness'] = brightness
df['B_sd'] = B_sd
df['red'] = red
df['blue'] = blue
df['green'] = green
df['color_var'] = color_var
df['c_moment'] = c_moment
df['contrast'] = contra
df['img_size'] = img_size
df['edge_intensity'] = edge_intensity
df['edge_num'] = edge_num
df['saturation'] = saturation
df['sift'] = sift
df['psnr'] = psnr
df['centroid_l'] = centroid_l

In [ ]:
# 새로운 열 생성
df[['c_moment_x', 'c_moment_y']] = pd.DataFrame(df['c_moment'].tolist(), index=df.index)

# 기존 열 삭제
# df = df.drop('c_moment', axis=1)

In [ ]:
df['bbox_prop'] = df['bbox_size']/df['img_size'].values

In [ ]:
df = df[['image_file'] + [col for col in df.columns if col != 'image_file']]

In [ ]:
tmp = df.sort_values(by='score', ascending=False).reset_index(drop=True)

In [ ]:
tmp.columns

Index(['image_file', 'center', 'scale', 'score', 'bbox', 'width', 'height',
       'bbox_size', 'x', 'y', 'entropy', 'laplacian', 'brightness', 'B_sd',
       'red', 'blue', 'green', 'color_var', 'c_moment', 'contrast', 'img_size',
       'edge_intensity', 'edge_num', 'saturation', 'sift', 'psnr',
       'centroid_l', 'c_moment_x', 'c_moment_y', 'bbox_prop'],
      dtype='object')

In [ ]:
# tmp.to_csv("refine_merged_df.csv", index=False)

-----------

### Haralick feature

In [ ]:
import mahotas

image_file = []
haralick = []
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]

    # Haralick 텍스처 특징 계산
    features = mahotas.features.haralick(cropped_image).mean(axis=0)
    haralick.append(features)
    image_file.append(image_file_path)

In [ ]:
# 각 Haralick 특징을 열로 하는 데이터프레임 생성
haralick_df = pd.DataFrame(haralick, columns=[
    'ASM', 'Contrast', 'Correlation', 'Variance', 'IDM', 'Sum_Average',
    'Sum_Variance', 'Sum_Entropy', 'Entropy', 'Difference_Variance',
    'Difference_Entropy', 'IMC1', 'IMC2'
])

# 데이터프레임 출력
print(haralick_df)

           ASM     Contrast  Correlation     Variance       IDM  Sum_Average  \
0     0.000609   646.281536     0.889691  2929.366619  0.197690   188.516126   
1     0.000986   753.491949     0.947918  7232.684350  0.230113   301.534133   
2     0.000611   573.337685     0.939569  4743.964206  0.175806   277.066613   
3     0.011206   132.208872     0.886605   582.616944  0.485988    54.347856   
4     0.002986   457.923017     0.931123  3324.605567  0.317951   227.930042   
...        ...          ...          ...          ...       ...          ...   
6329  0.000398   231.210898     0.979381  5606.753123  0.247869   269.102102   
6330  0.000531   718.349006     0.849774  2388.876832  0.157800   250.971430   
6331  0.001080   174.553447     0.984492  5627.147242  0.270758   236.624732   
6332  0.000324  1086.098211     0.852492  3681.542436  0.164291   302.343720   
6333  0.000961   273.519083     0.967100  4155.920027  0.315945   301.266977   

      Sum_Variance  Sum_Entropy    Entr

In [ ]:
tmp = pd.concat([df, haralick_df], axis=1)
tmp = tmp.sort_values(by='score', ascending=False).reset_index(drop=True)

In [ ]:
numeric_cols = tmp.select_dtypes(include=['float64', 'int64']).columns
numeric_df = tmp[numeric_cols]
numeric_df

,score,width,height,bbox_size,x,y,entropy,laplacian,brightness,B_sd,...,Variance,IDM,Sum_Average,Sum_Variance,Sum_Entropy,Entropy,Difference_Variance,Difference_Entropy,IMC1,IMC2
0,0.960156,175.78,365.92,64321.4176,231.39,23.32,6.991802,2396.599312,109.016344,55.437876,...,3085.183045,0.272328,218.321967,11805.860404,7.976299,11.780171,0.000351,4.483578,-0.316311,0.993586
1,0.946107,175.98,269.79,47477.6442,222.64,85.72,7.640442,4402.490432,136.528625,57.042084,...,3246.922688,0.086307,272.560296,12048.803662,8.617964,13.766669,0.000097,5.728885,-0.197973,0.974561
2,0.945302,247.26,443.74,109719.1524,39.11,48.82,7.261376,1511.846484,138.665786,52.863906,...,2796.926974,0.255842,277.354480,10746.326177,8.235842,12.028241,0.000368,4.271601,-0.343483,0.996333
3,0.942805,125.15,386.46,48365.4690,195.29,211.79,7.541892,7403.016087,98.924808,55.624675,...,3097.140189,0.101048,198.059275,11180.774590,8.463239,13.577417,0.000113,5.674594,-0.199670,0.974415
4,0.941582,131.46,228.37,30021.5202,274.43,78.75,7.315833,3676.558615,153.935550,39.944085,...,1602.795150,0.084901,307.765087,5720.189042,8.223095,13.280933,0.000107,5.537443,-0.185451,0.964452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,0.213297,47.54,96.43,4584.2822,406.07,0.00,6.181008,663.084400,61.003103,33.619329,...,1128.807063,0.194234,123.457601,4328.662984,7.114964,10.553525,0.000480,4.492450,-0.296937,0.986859
6330,0.212928,80.53,135.90,10944.0270,0.00,0.00,4.793607,40.994660,35.096204,63.665830,...,3945.478955,0.519975,68.458457,15714.912591,5.674219,7.333457,0.001055,2.768688,-0.457672,0.993321
6331,0.201631,69.67,65.00,4528.5500,408.33,362.50,7.348773,3286.761896,97.960981,43.993721,...,1928.911575,0.099488,196.412494,7113.236356,8.251805,12.176833,0.000166,5.272366,-0.342831,0.996724
6332,0.000000,54.45,68.97,3755.4165,228.50,91.80,5.477101,498.482781,22.003268,14.538881,...,207.225784,0.230318,44.017930,739.038855,6.378352,9.595479,0.000745,3.851126,-0.246428,0.964953


In [ ]:
tmp2 = numeric_df.drop(columns=['x','y'])
tmp2

,score,width,height,bbox_size,entropy,laplacian,brightness,B_sd,red,blue,...,Variance,IDM,Sum_Average,Sum_Variance,Sum_Entropy,Entropy,Difference_Variance,Difference_Entropy,IMC1,IMC2
0,0.960156,175.78,365.92,64321.4176,6.991802,2396.599312,109.016344,55.437876,89.456751,116.950716,...,3085.183045,0.272328,218.321967,11805.860404,7.976299,11.780171,0.000351,4.483578,-0.316311,0.993586
1,0.946107,175.98,269.79,47477.6442,7.640442,4402.490432,136.528625,57.042084,143.867764,132.133001,...,3246.922688,0.086307,272.560296,12048.803662,8.617964,13.766669,0.000097,5.728885,-0.197973,0.974561
2,0.945302,247.26,443.74,109719.1524,7.261376,1511.846484,138.665786,52.863906,129.274911,145.894198,...,2796.926974,0.255842,277.354480,10746.326177,8.235842,12.028241,0.000368,4.271601,-0.343483,0.996333
3,0.942805,125.15,386.46,48365.4690,7.541892,7403.016087,98.924808,55.624675,94.581389,78.445679,...,3097.140189,0.101048,198.059275,11180.774590,8.463239,13.577417,0.000113,5.674594,-0.199670,0.974415
4,0.941582,131.46,228.37,30021.5202,7.315833,3676.558615,153.935550,39.944085,155.740893,157.968963,...,1602.795150,0.084901,307.765087,5720.189042,8.223095,13.280933,0.000107,5.537443,-0.185451,0.964452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,0.213297,47.54,96.43,4584.2822,6.181008,663.084400,61.003103,33.619329,63.007535,65.351950,...,1128.807063,0.194234,123.457601,4328.662984,7.114964,10.553525,0.000480,4.492450,-0.296937,0.986859
6330,0.212928,80.53,135.90,10944.0270,4.793607,40.994660,35.096204,63.665830,31.542130,40.154444,...,3945.478955,0.519975,68.458457,15714.912591,5.674219,7.333457,0.001055,2.768688,-0.457672,0.993321
6331,0.201631,69.67,65.00,4528.5500,7.348773,3286.761896,97.960981,43.993721,106.094537,103.439242,...,1928.911575,0.099488,196.412494,7113.236356,8.251805,12.176833,0.000166,5.272366,-0.342831,0.996724
6332,0.000000,54.45,68.97,3755.4165,5.477101,498.482781,22.003268,14.538881,27.245915,15.398148,...,207.225784,0.230318,44.017930,739.038855,6.378352,9.595479,0.000745,3.851126,-0.246428,0.964953


In [ ]:
not_normalized_df = pd.concat([df['image_file'], tmp2], axis=1)
not_normalized_df.to_csv("df_0103.csv", index=False)


In [ ]:
tmp2 = pd.concat([not_normalized_df, tmp['bbox']], axis=1)
tmp2.to_csv("df_0103_added_bbox.csv", index=False)

### normalized

In [ ]:
# numeric_cols = tmp.select_dtypes(include=['float64', 'int64']).columns
# numeric_df = tmp[numeric_cols]
# df_normalized = (numeric_df - numeric_df.min()) / (numeric_df.max() - numeric_df.min())

In [ ]:
# df_normalized.columns

Index(['score', 'width', 'height', 'bbox_size', 'x', 'y', 'entropy',
       'laplacian', 'brightness', 'B_sd', 'red', 'blue', 'green', 'color_var',
       'contrast', 'img_size', 'edge_intensity', 'saturation', 'sift',
       'c_moment_x', 'c_moment_y', 'bbox_prop', 'ASM', 'Contrast',
       'Correlation', 'Variance', 'IDM', 'Sum_Average', 'Sum_Variance',
       'Sum_Entropy', 'Entropy', 'Difference_Variance', 'Difference_Entropy',
       'IMC1', 'IMC2'],
      dtype='object')

In [ ]:
# normalized_ver
# pd.concat([tmp['image_file'], df_normalized], axis=1).to_csv("n_df.csv", index=False)


- Angular Second Moment (ASM): 에지 강도의 정도를 나타내며, 값이 작을수록 텍스처가 덜 일정하거나 변화가 많음을 나타냅니다.

- Contrast: 인접한 픽셀 값의 대비를 나타냅니다. 값이 클수록 이미지의 대비가 뚜렷하고 선명함을 의미합니다.

- Correlation: 이미지에서 한 방향의 밝기 변화가 다른 방향으로 얼마나 유사한지를 나타내며, 1에 가까울수록 밝기 변화가 일정합니다.

- Sum of Squares (Variance): 픽셀 값의 분산을 나타냅니다. 값이 클수록 이미지의 텍스처가 다양하게 분포되어 있음을 나타냅니다.

- Inverse Difference Moment (IDM): 픽셀 값의 변화가 얼마나 부드럽게 일어나는지를 나타내며, 값이 클수록 부드러운 텍스처를 의미합니다.

- Sum Average: 텍스처에서 발생한 픽셀 값의 합의 평균을 나타냅니다.

- Sum Variance: 픽셀 값의 합의 분산을 나타냅니다.

- Sum Entropy: 텍스처의 복잡성을 나타내며, 값이 클수록 텍스처가 복잡하다는 것을 의미합니다.

- Entropy: 이미지의 무질서도를 나타냅니다. 값이 클수록 이미지가 무질서하다는 것을 의미합니다.

- Difference Variance: 픽셀 값 간의 차이의 분산을 나타냅니다.

- Difference Entropy: 픽셀 값 간의 차이의 무질서도를 나타냅니다.

- Information Measures of Correlation 1 (IMC1): Correlation에 기반한 정보의 양을 나타냅니다.

- Information Measures of Correlation 2 (IMC2): Correlation에 기반한 정보의 양을 나타냅니다.


In [80]:
origin_df = pd.read_csv("..\dataset\df_0129_added_bbox.csv")
origin_df.columns

Index(['image_file', 'score', 'width', 'height', 'bbox_size', 'entropy',
       'laplacian', 'brightness', 'B_sd', 'red', 'blue', 'green', 'color_var',
       'contrast', 'img_size', 'edge_intensity', 'saturation', 'sift',
       'centroid_l', 'c_moment_x', 'c_moment_y', 'bbox_prop', 'ASM',
       'Contrast', 'Correlation', 'Variance', 'IDM', 'Sum_Average',
       'Sum_Variance', 'Sum_Entropy', 'Entropy', 'Difference_Variance',
       'Difference_Entropy', 'IMC1', 'IMC2', 'bbox'],
      dtype='object')

In [83]:
# Assuming 'bbox' column contains string representations of lists
origin_df['bbox'] = origin_df['bbox'].apply(ast.literal_eval)

# Now you can calculate the center and add the 'center' column
origin_df['center'] = origin_df['bbox'].apply(lambda box: [(box[0] + box[2]) / 2, (box[1] + box[3]) / 2, box[2], box[3]])

In [101]:
origin_df['image_id']= origin_df['image_file'].str[-16:-4].astype(int)

In [102]:
result_df

,image_id,center,score
0,136,"[33.959999084472656, 217.1750030517578]",0.615619
1,136,"[27.110000610351562, 243.05999755859375]",0.548477
2,397,"[116.06999969482422, 102.16500091552734]",0.422394
3,443,"[377.0249938964844, 236.80499267578125]",0.445117
4,443,"[73.57499694824219, 308.0450134277344]",0.430084
...,...,...,...
11732,581674,"[159.0500030517578, 222.47999572753906]",0.432098
11733,581683,"[403.7650146484375, 490.3500061035156]",0.601338
11734,581683,"[381.5, 430.114990234375]",0.467200
11735,581880,"[111.37000274658203, 122.22000122070312]",0.273316


## metric 계산 적용

In [8]:
entropy = []
laplacian = []
brightness = []
B_sd = []
c_moment = []
contra = []
img_size = []
edge_intensity = []
edge_num = []
sift = []
psnr = []
centroid_l = []

for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    height, width= image.shape
    img_size_value = width * height

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    entropy_value = calculate_image_entropy_(cropped_image)
    laplacian_value = calculate_laplacian_variance_(cropped_image)
    brightness_value = np.mean(cropped_image)
    sd_value = np.std(cropped_image)
    c_moment_value = moment(cropped_image)
    contrast_value = calculate_contrast(cropped_image)
    edge_num_value, edge_value = edge(cropped_image)
    SIFT_count_value = SIFT_count(cropped_image)
    psnr_value = calculate_psnr(cropped_image)
    cx, cy = c_moment_value
    centroid_l_value = centroid_degree(w, h, cx, cy)

    entropy.append(entropy_value)
    laplacian.append(laplacian_value)
    brightness.append(brightness_value)
    B_sd.append(sd_value)
    c_moment.append(c_moment_value)
    contra.append(contrast_value)
    img_size.append(img_size_value)
    edge_intensity.append(edge_value)
    edge_num.append(edge_num_value)
    sift.append(SIFT_count_value)
    psnr.append(psnr_value)
    centroid_l.append(centroid_l_value)

In [9]:
# RGB
red = []
green = []
blue = []
color_var = []

# Not Grayscale
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]
    
    # 이미지 배열에서 각 채널의 값 가져오기
    red_channel = cropped_image[:, :, 0]
    green_channel = cropped_image[:, :, 1]
    blue_channel = cropped_image[:, :, 2]

    # 각 채널별 평균과 분산 계산
    red_mean, red_variance = np.mean(red_channel), np.var(red_channel)
    green_mean, green_variance = np.mean(green_channel), np.var(green_channel)
    blue_mean, blue_variance = np.mean(blue_channel), np.var(blue_channel)
    variance = np.mean([red_variance,green_variance,blue_variance])

    red.append(red_mean)
    green.append(green_mean)
    blue.append(blue_mean)
    color_var.append(variance)

In [10]:
# HSV
saturation = []

# Not Grayscale
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]

    saturation_value = np.mean(cropped_image[:, :, 1])

    saturation.append(saturation_value)

In [11]:
df['entropy'] = entropy
df['laplacian'] = laplacian
df['brightness'] = brightness
df['B_sd'] = B_sd
df['red'] = red
df['blue'] = blue
df['green'] = green
df['color_var'] = color_var
df['c_moment'] = c_moment
df['contrast'] = contra
df['img_size'] = img_size
df['edge_intensity'] = edge_intensity
df['edge_num'] = edge_num
df['saturation'] = saturation
df['sift'] = sift
df['psnr'] = psnr
df['centroid_l'] = centroid_l

In [12]:
# 새로운 열 생성
df[['c_moment_x', 'c_moment_y']] = pd.DataFrame(df['c_moment'].tolist(), index=df.index)

# 기존 열 삭제
# df = df.drop('c_moment', axis=1)

In [13]:
df['bbox_prop'] = df['bbox_size']/df['img_size'].values

In [14]:
df = df[['image_file'] + [col for col in df.columns if col != 'image_file']]

In [15]:
tmp = df.sort_values(by='score', ascending=False).reset_index(drop=True)

In [16]:
tmp.columns

Index(['image_file', 'center', 'scale', 'score', 'bbox', 'width', 'height',
       'bbox_size', 'x', 'y', 'entropy', 'laplacian', 'brightness', 'B_sd',
       'red', 'blue', 'green', 'color_var', 'c_moment', 'contrast', 'img_size',
       'edge_intensity', 'edge_num', 'saturation', 'sift', 'psnr',
       'centroid_l', 'c_moment_x', 'c_moment_y', 'bbox_prop'],
      dtype='object')

In [17]:
# tmp.to_csv("refine_merged_df.csv", index=False)

-----------

### Haralick feature

In [18]:
import mahotas

image_file = []
haralick = []
for i in range(len(df)):
    bbox = df.loc[i,'bbox']
    
    # bbox 정보에서 x, y, w, h 추출
    x, y, w, h = map(int, bbox)

    # 이미지 로드
    image_file_path = df.loc[i,'image_file']
    image = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)

    # 이미지를 bbox에 맞게 자르기
    cropped_image = image[y:y+h, x:x+w]

    # Haralick 텍스처 특징 계산
    features = mahotas.features.haralick(cropped_image).mean(axis=0)
    haralick.append(features)
    image_file.append(image_file_path)

In [19]:
# 각 Haralick 특징을 열로 하는 데이터프레임 생성
haralick_df = pd.DataFrame(haralick, columns=[
    'ASM', 'Contrast', 'Correlation', 'Variance', 'IDM', 'Sum_Average',
    'Sum_Variance', 'Sum_Entropy', 'Entropy', 'Difference_Variance',
    'Difference_Entropy', 'IMC1', 'IMC2'
])

# 데이터프레임 출력
print(haralick_df)

           ASM     Contrast  Correlation     Variance       IDM  Sum_Average  \
0     0.000609   646.281536     0.889691  2929.366619  0.197690   188.516126   
1     0.000986   753.491949     0.947918  7232.684350  0.230113   301.534133   
2     0.000611   573.337685     0.939569  4743.964206  0.175806   277.066613   
3     0.011206   132.208872     0.886605   582.616944  0.485988    54.347856   
4     0.002986   457.923017     0.931123  3324.605567  0.317951   227.930042   
...        ...          ...          ...          ...       ...          ...   
6329  0.000398   231.210898     0.979381  5606.753123  0.247869   269.102102   
6330  0.000531   718.349006     0.849774  2388.876832  0.157800   250.971430   
6331  0.001080   174.553447     0.984492  5627.147242  0.270758   236.624732   
6332  0.000324  1086.098211     0.852492  3681.542436  0.164291   302.343720   
6333  0.000961   273.519083     0.967100  4155.920027  0.315945   301.266977   

      Sum_Variance  Sum_Entropy    Entr

In [20]:
tmp = pd.concat([df, haralick_df], axis=1)
tmp = tmp.sort_values(by='score', ascending=False).reset_index(drop=True)

In [21]:
numeric_cols = tmp.select_dtypes(include=['float64', 'int64']).columns
numeric_df = tmp[numeric_cols]
numeric_df

,score,width,height,bbox_size,x,y,entropy,laplacian,brightness,B_sd,...,Variance,IDM,Sum_Average,Sum_Variance,Sum_Entropy,Entropy,Difference_Variance,Difference_Entropy,IMC1,IMC2
0,0.960156,175.78,365.92,64321.4176,231.39,23.32,6.991802,2396.599312,109.016344,55.437876,...,3085.183045,0.272328,218.321967,11805.860404,7.976299,11.780171,0.000351,4.483578,-0.316311,0.993586
1,0.946107,175.98,269.79,47477.6442,222.64,85.72,7.640442,4402.490432,136.528625,57.042084,...,3246.922688,0.086307,272.560296,12048.803662,8.617964,13.766669,0.000097,5.728885,-0.197973,0.974561
2,0.945302,247.26,443.74,109719.1524,39.11,48.82,7.261376,1511.846484,138.665786,52.863906,...,2796.926974,0.255842,277.354480,10746.326177,8.235842,12.028241,0.000368,4.271601,-0.343483,0.996333
3,0.942805,125.15,386.46,48365.4690,195.29,211.79,7.541892,7403.016087,98.924808,55.624675,...,3097.140189,0.101048,198.059275,11180.774590,8.463239,13.577417,0.000113,5.674594,-0.199670,0.974415
4,0.941582,131.46,228.37,30021.5202,274.43,78.75,7.315833,3676.558615,153.935550,39.944085,...,1602.795150,0.084901,307.765087,5720.189042,8.223095,13.280933,0.000107,5.537443,-0.185451,0.964452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,0.213297,47.54,96.43,4584.2822,406.07,0.00,6.181008,663.084400,61.003103,33.619329,...,1128.807063,0.194234,123.457601,4328.662984,7.114964,10.553525,0.000480,4.492450,-0.296937,0.986859
6330,0.212928,80.53,135.90,10944.0270,0.00,0.00,4.793607,40.994660,35.096204,63.665830,...,3945.478955,0.519975,68.458457,15714.912591,5.674219,7.333457,0.001055,2.768688,-0.457672,0.993321
6331,0.201631,69.67,65.00,4528.5500,408.33,362.50,7.348773,3286.761896,97.960981,43.993721,...,1928.911575,0.099488,196.412494,7113.236356,8.251805,12.176833,0.000166,5.272366,-0.342831,0.996724
6332,0.000000,54.45,68.97,3755.4165,228.50,91.80,5.477101,498.482781,22.003268,14.538881,...,207.225784,0.230318,44.017930,739.038855,6.378352,9.595479,0.000745,3.851126,-0.246428,0.964953


In [22]:
tmp2 = numeric_df.drop(columns=['x','y'])
tmp2

,score,width,height,bbox_size,entropy,laplacian,brightness,B_sd,red,blue,...,Variance,IDM,Sum_Average,Sum_Variance,Sum_Entropy,Entropy,Difference_Variance,Difference_Entropy,IMC1,IMC2
0,0.960156,175.78,365.92,64321.4176,6.991802,2396.599312,109.016344,55.437876,89.456751,116.950716,...,3085.183045,0.272328,218.321967,11805.860404,7.976299,11.780171,0.000351,4.483578,-0.316311,0.993586
1,0.946107,175.98,269.79,47477.6442,7.640442,4402.490432,136.528625,57.042084,143.867764,132.133001,...,3246.922688,0.086307,272.560296,12048.803662,8.617964,13.766669,0.000097,5.728885,-0.197973,0.974561
2,0.945302,247.26,443.74,109719.1524,7.261376,1511.846484,138.665786,52.863906,129.274911,145.894198,...,2796.926974,0.255842,277.354480,10746.326177,8.235842,12.028241,0.000368,4.271601,-0.343483,0.996333
3,0.942805,125.15,386.46,48365.4690,7.541892,7403.016087,98.924808,55.624675,94.581389,78.445679,...,3097.140189,0.101048,198.059275,11180.774590,8.463239,13.577417,0.000113,5.674594,-0.199670,0.974415
4,0.941582,131.46,228.37,30021.5202,7.315833,3676.558615,153.935550,39.944085,155.740893,157.968963,...,1602.795150,0.084901,307.765087,5720.189042,8.223095,13.280933,0.000107,5.537443,-0.185451,0.964452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6329,0.213297,47.54,96.43,4584.2822,6.181008,663.084400,61.003103,33.619329,63.007535,65.351950,...,1128.807063,0.194234,123.457601,4328.662984,7.114964,10.553525,0.000480,4.492450,-0.296937,0.986859
6330,0.212928,80.53,135.90,10944.0270,4.793607,40.994660,35.096204,63.665830,31.542130,40.154444,...,3945.478955,0.519975,68.458457,15714.912591,5.674219,7.333457,0.001055,2.768688,-0.457672,0.993321
6331,0.201631,69.67,65.00,4528.5500,7.348773,3286.761896,97.960981,43.993721,106.094537,103.439242,...,1928.911575,0.099488,196.412494,7113.236356,8.251805,12.176833,0.000166,5.272366,-0.342831,0.996724
6332,0.000000,54.45,68.97,3755.4165,5.477101,498.482781,22.003268,14.538881,27.245915,15.398148,...,207.225784,0.230318,44.017930,739.038855,6.378352,9.595479,0.000745,3.851126,-0.246428,0.964953


In [23]:
not_normalized_df = pd.concat([df['image_file'], tmp2], axis=1)
not_normalized_df.to_csv("df_0103.csv", index=False)


In [11]:
tmp2 = pd.concat([not_normalized_df, tmp['bbox']], axis=1)
tmp2.to_csv("df_0103_added_bbox.csv", index=False)

### normalized

In [58]:
# numeric_cols = tmp.select_dtypes(include=['float64', 'int64']).columns
# numeric_df = tmp[numeric_cols]
# df_normalized = (numeric_df - numeric_df.min()) / (numeric_df.max() - numeric_df.min())

In [61]:
# df_normalized.columns

Index(['score', 'width', 'height', 'bbox_size', 'x', 'y', 'entropy',
       'laplacian', 'brightness', 'B_sd', 'red', 'blue', 'green', 'color_var',
       'contrast', 'img_size', 'edge_intensity', 'saturation', 'sift',
       'c_moment_x', 'c_moment_y', 'bbox_prop', 'ASM', 'Contrast',
       'Correlation', 'Variance', 'IDM', 'Sum_Average', 'Sum_Variance',
       'Sum_Entropy', 'Entropy', 'Difference_Variance', 'Difference_Entropy',
       'IMC1', 'IMC2'],
      dtype='object')

In [60]:
# normalized_ver
# pd.concat([tmp['image_file'], df_normalized], axis=1).to_csv("n_df.csv", index=False)


- Angular Second Moment (ASM): 에지 강도의 정도를 나타내며, 값이 작을수록 텍스처가 덜 일정하거나 변화가 많음을 나타냅니다.

- Contrast: 인접한 픽셀 값의 대비를 나타냅니다. 값이 클수록 이미지의 대비가 뚜렷하고 선명함을 의미합니다.

- Correlation: 이미지에서 한 방향의 밝기 변화가 다른 방향으로 얼마나 유사한지를 나타내며, 1에 가까울수록 밝기 변화가 일정합니다.

- Sum of Squares (Variance): 픽셀 값의 분산을 나타냅니다. 값이 클수록 이미지의 텍스처가 다양하게 분포되어 있음을 나타냅니다.

- Inverse Difference Moment (IDM): 픽셀 값의 변화가 얼마나 부드럽게 일어나는지를 나타내며, 값이 클수록 부드러운 텍스처를 의미합니다.

- Sum Average: 텍스처에서 발생한 픽셀 값의 합의 평균을 나타냅니다.

- Sum Variance: 픽셀 값의 합의 분산을 나타냅니다.

- Sum Entropy: 텍스처의 복잡성을 나타내며, 값이 클수록 텍스처가 복잡하다는 것을 의미합니다.

- Entropy: 이미지의 무질서도를 나타냅니다. 값이 클수록 이미지가 무질서하다는 것을 의미합니다.

- Difference Variance: 픽셀 값 간의 차이의 분산을 나타냅니다.

- Difference Entropy: 픽셀 값 간의 차이의 무질서도를 나타냅니다.

- Information Measures of Correlation 1 (IMC1): Correlation에 기반한 정보의 양을 나타냅니다.

- Information Measures of Correlation 2 (IMC2): Correlation에 기반한 정보의 양을 나타냅니다.
